In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm
import pandas as pd
import threading
import psutil, multiprocessing
from os import walk

#В этой части при помощи selenium находим адреса на google-картах и сохраняем координаты (в сыром виде, как ссылки)

In [3]:
def make_driver():
    option = Options()
    option.headless = True
    option.add_argument("--disable-gpu")  
    prefs = {'profile.default_content_setting_values': {'images':2, 'javascript':2}}
    driver = webdriver.Chrome(".\\chromedriver.exe", options=option)
    driver.get("about:blank")
    return driver

In [4]:
#Функция, которая выдаёт txt-файл с координатами (в сыром виде)

URLsearch = lambda x: f"https://www.google.com/maps/search/{x}"

def one_task(values, driver_unformed):
    driver = driver_unformed() #Передадим сюда функцию, которая запускает драйвер 
    for v in values: #В values передадим список с частью строк из df
        try:
            id, address = v
            driver.get(URLsearch(address))
            
            #Взяла селектор CSS из статьи
            #https://medium.com/nuances-of-programming/python-selenium-как-получить-координаты-по-адресам-ea7a78ffdc0d
            for c in driver.find_elements(By.CSS_SELECTOR, 'meta[itemprop=image]'):
                #Заранее создаём папку nts
                #Создаём в ней txt файл и записываем туда то, что нашли по CSS-селектору
                with open(f"nts\\{id}.txt", "w", encoding="utf-8") as f:
                    f.write(f"{c.get_attribute('content')}")
                break
        except Exception as e:
            print(f"ERROR: {e}")
    driver.close()

In [5]:
df = pd.read_csv('united_reestr.csv')[['id', 'address']].drop_duplicates()
df

,id,address
0,9001067,"Респ. Адыгея, г. Адыгейск, д. 888888"
1,6472757,"Респ. Адыгея, г. Адыгейск, ул. Горького, д. 25"
2,6472669,"Респ. Адыгея, г. Адыгейск, ул. Горького, д. 27"
3,8928733,"Респ. Адыгея, г. Адыгейск, ул. Горького, д. 31"
4,6551241,"Респ. Адыгея, г. Адыгейск, ул. Коммунистическа..."
...,...,...
828004,9103545,"г. Севастополь, ул. Ярославская, д. 30"
828005,9103546,"г. Севастополь, ул. Ярославская, д. 33"
828006,9103547,"г. Севастополь, ул. Ярославская, д. 40"
828007,9098909,"г. Севастополь, ул. Ясная, д. 1, лит. б"


In [6]:
#Пропускаем строки, которые уже обработали 

#Возвращаем содержимое файла n, чтобы записать потом в csv-файл и удалить txt
def skipped_f(n):
    with open(f"nts\\{n}", encoding="utf-8") as f:
        return ''.join(f.readlines())

#Создаём df с id и url(в котором есть координаты) и записываем его в файл.
#Возвращаем df_skip, чтобы потом взять из него id, которые уже обработали
def skipped_to_df(first_time=False):
    
    if not first_time:
        df_already = pd.read_csv("nts.csv")
    
    for i,j,k in walk('nts'):
        break
    df_files = pd.DataFrame(
        #Получаем из названия файла id, далее получаем содержимое файла - добавляем в кортеж
        [(int(k0.replace(".txt", "")), skipped_f(k0)) for k0 in k],
        columns = ["id", "url"] #внутри url лежат координаты
    )
    
    #Если запускаем не первый раз - добавляем к предыдущему результату
    df_skip = df_files if first_time else pd.merge(df_already, df_files, how="outer")
    
    #Записываем в файл (или перезаписываем)
    df_skip.to_csv("nts.csv", index=False)
    return df_skip


# def skipper_oldver():
#     for i,j,k in walk('nts'):
#         break
#     return [int(k0.replace(".txt", "")) for k0 in k]

skipper = lambda: skipped_to_df()['id'].tolist()

In [62]:
#Подготовка потоков к запуску

LIMIT = 1000 #133312 #26656 #113760 - количество строк, которе запускалось за один раз
SYS_THREADS = 16 #количество ядер
TIMES_PER_THREAD = 2 #сколько потоков на ядро

THREAD_LIMIT = SYS_THREADS * TIMES_PER_THREAD

# 1:45 for 100/4, 8:00 for 1000/8, 5:00 for 1000/16, 4:31 for 1000/32, 4:58 for 1000/(16 * 3)

#сколько строк на каждый поток (в теории)
IN_LIMIT = LIMIT // THREAD_LIMIT 

skips = skipper() 
unskipped = df[~df['id'].isin(skips)]
unskipped_ids = unskipped['id'].tolist()[:LIMIT]
unskipped_addrs = unskipped['address'].tolist()[:LIMIT]

#сколько строк на каждый поток
per = max(1, len(unskipped_ids) // THREAD_LIMIT) 
pairs = list(zip(unskipped_ids, unskipped_addrs)) 

#списки по 32 элемента (т.к. у нас 32 потока)
values = [pairs[i:i+per] for i in range(0, len(pairs), per)][:THREAD_LIMIT] 

threads = [threading.Thread(target=one_task, args=(v, make_driver)) for v in values]

#Хотим понять, сколько закачается за ночь
stat1, stat2 = len(values), len(values[0]) if values else 0
stat_total = stat1 * stat2
stat_skipped = len(unskipped_ids) - stat_total 
(f"{stat1} threads", f"{stat2} tasks per thread", f"{stat_total} total ({stat_skipped} skipped)")

('0 threads', '0 tasks per thread', '0 total (0 skipped)')

In [61]:
%%time
#Запускаем вот это вот всё

SKIP_IT = False #Чтобы случайно не запустить ячейку

if not SKIP_IT:
    [thread.start() for thread in threads]
    [thread.join() for thread in threads]

print()

C:\Users\spopkov\AppData\Local\Temp\ipykernel_2000\3630013382.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(".\\chromedriver.exe", options=option)



CPU times: total: 438 ms
Wall time: 9.03 s


In [58]:
!taskkill /im chromedriver.exe /f

ERROR: The process "chromedriver.exe" not found.


In [42]:
#Хотим понять, сколько закачается за ночь

HOURS = 10

mintasks = int((1000 * 60 / 4.5) * HOURS + 1) // 32 * 32
maxtasks = (128000 // 9 * HOURS + 1) // 32 * 32 #128000 - реально закачалось за один раз
f"{HOURS} hours: between {mintasks} - {maxtasks}"

'10 hours: between 133312 - 142208'

In [63]:
len(set(skips))

828006